In [39]:
from glob import glob
import numpy as np

In [40]:
stripfiles = sorted(glob('meshes/frame*.obj'))
stripfiles = [fl for fl in stripfiles if 't1' not in fl]
print('    ', len(stripfiles))
print(stripfiles[0])
print(stripfiles[-1])

     288
meshes\frame30000_strip_o1001_s36_572.obj
meshes\frame30000_strip_o994_s36_851.obj


In [41]:
def parsev(line, comment=True, dtype=float):
    if comment: line = line.replace('# ', '')
    parts = line.split()
    token = parts[0]
    vlist = [dtype(val) for val in parts[1:]]
    return vlist, token

parsev('# asdf 1 2 3 4')

([1.0, 2.0, 3.0, 4.0], 'asdf')

In [43]:
RSTRIDE = 3
BSTART = 40
def load_markup(fname):
    with open(fname) as fl:
        raw = fl.read()
    lines = [ln for ln in raw.split('\n') if ln]
    registers = lines[1:1+256]
    registers = [parsev(ln)[0] for ln in registers]
    
    vstart, vend, ventries = -1, -1, 1
    vstart = lines.index('# Data:') + 1
    for lii, line in enumerate(lines[vstart:]):
        if line[0] == 'v':
            ventries = lii + 1
            break
    for lii, line in enumerate(lines[vstart:]):
        if line[0] == 'f':
            vend = lii
            break
    vend += vstart
#     print(vstart, vend, ventries)
        
    vgrouped = [lines[lii:lii+ventries] for lii in range(vstart, vend, ventries)]
    faces = [parsev(ln, comment=False, dtype=int)[0] for ln in lines if ln[0] == 'f']
    
    verts = []
    for group in vgrouped:
        ent = {}
        converted = [parsev(line) for line in group[:ventries-1]]
        for vector, token in converted: ent[token] = vector
        
        vertex, _ = parsev(group[-1], comment=False)
        ent['vertex'] = vertex + [1,]
        verts.append(ent)

    for ent in verts:
        vertex = ent['vertex']
        if 'blend' in ent:
            bind = int(ent['blend'][0])
            assert bind >= 0
            boffset = BSTART + RSTRIDE * bind
            bvect = np.zeros(4)
            for dim in range(3):
                bweights = np.array(registers[boffset+dim])
                bvect[dim] = np.dot(vertex, bweights)
            bvect[3] = 1
            ent['world'] = bvect
        else:
            ent['world'] = ent['vertex']
    
    return verts, faces, registers

verts, faces, regs = load_markup(stripfiles[0])
# verts, faces, regs = load_markup('meshes/frame30000_strip_o149_s60_2608.obj')
print(verts[0]['world'])
print(verts[-1]['world'])
print(faces[0])

[4208.003418, 433.771393, 11131.350586, 1]
[5170.827637, 390.140747, 11579.664063, 1]
[1, 2, 3]


In [44]:
def writeobj(fname, verts, faces):
    with open(fname, 'w') as fl:
        for ent in verts:
            fl.write('v %s\n' % ' '.join(['%f' % val for val in ent['world'][:3]]))
        for fdef in faces:
            fl.write('f %s\n' % ' '.join(['%d' % val for val in fdef]))

stripfiles = sorted(glob('meshes/frame*.obj'))
stripfiles = [fl for fl in stripfiles if 't1' not in fl]

for fname in stripfiles:
    verts, faces, regs = load_markup(fname)
    fname = fname.replace('frame', 'blendf').replace('meshes', 'outputs')
#     print(fname)
    writeobj(fname, verts, faces)

In [67]:
markupfiles = sorted(glob('meshes/frame*.obj'))
print(len(markupfiles))

class merged:
    verts = []
    faces = []
    
foffset = 0
for fname in markupfiles:
    verts, faces, regs = load_markup(fname)
    merged.verts += verts
    
#     print(fname, faces[0])
    offset_faces = []
    for face in faces:
        offset_faces += [[val + foffset for val in face]]
    assert len(offset_faces) == len(faces)
    merged.faces += offset_faces
    
    foffset += len(verts)

with open('blended.obj', 'w') as fl:
    for vert in merged.verts:
#         print(vert)
        fl.write('v %s\n' % ' '.join(['%f' % val for val in vert['world'][:3]]))
    for face in merged.faces:
        fl.write('f %s\n' % ' '.join(['%d' % val for val in face]))
        
with open('merged.obj', 'w') as fl:
    for vert in merged.verts:
#         print(vert)
        fl.write('v %s\n' % ' '.join(['%f' % val for val in vert['vertex'][:3]]))
    for face in merged.faces:
        fl.write('f %s\n' % ' '.join(['%d' % val for val in face]))
    

199


In [65]:
print(markupfiles[200:208])

['meshes\\frame30500_strip_o367_s36_1186.obj', 'meshes\\frame30500_strip_o370_s36_1566.obj', 'meshes\\frame30500_strip_o371_s36_1008.obj', 'meshes\\frame30500_strip_o372_s36_2733.obj', 'meshes\\frame30500_strip_o373_s36_2387.obj', 'meshes\\frame30500_strip_o380_s36_19697.obj', 'meshes\\frame30500_strip_o381_s36_720.obj', 'meshes\\frame30500_strip_o382_s36_9106.obj']
